In [1]:
import numpy as np
import pandas as pd
import time
import tqdm
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pymorphy2
import warnings
warnings.filterwarnings("ignore")

In [3]:
import glob
import os
import matplotlib
import artm
import nltk
import feedparser
from nltk.corpus import stopwords
"""nltk.download('stopwords')
nltk.download('punkt')
#nltk.download()"""

"nltk.download('stopwords')\nnltk.download('punkt')\n#nltk.download()"

In [4]:
pd.options.display.max_colwidth = 800

In [5]:
data = pd.read_csv('gender-tense_data.csv');
data = data.fillna('')

In [6]:
data.head(2)

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,text,past,pres,futr,male,female
0,0,J06.0,Острый ларингофарингит,29,2,,,,"На жевательной поверхности 2.6- кариозная полость средней глубины, выполненная пигментированным дентином. Зондирование слегка болезненно по эмалево-дентинной границе. Реакция на температурные раздражители отсутствует.Перкуссия б/б.",1,,0,0,0,0,0
1,1,N76.1,Подострый и хронический вагинит,45,2,,,считает себя больной на протяжении многих лет. Появилась боль и скованность в спине. По словам паицентки в настоящее время занимается конным спортом и появляется боль и скованность в поясничном отделе позвончника. В ближайшее время пациентка планирует беременность и поэтому обратилась к неврологу.,,1,считает себя больной на протяжении многих лет появилась боль и скованность в спине по словам паицентки в настоящее время занимается конным спортом и появляется боль и скованность в поясничном отделе позвончника в ближайшее время пациентка планирует беременность и поэтому обратилась к неврологу,2,4,0,0,2


In [9]:
df = data
df.head()

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,text,past,pres,futr,male,female
0,0,J06.0,Острый ларингофарингит,29,2,,,,"На жевательной поверхности 2.6- кариозная полость средней глубины, выполненная пигментированным дентином. Зондирование слегка болезненно по эмалево-дентинной границе. Реакция на температурные раздражители отсутствует.Перкуссия б/б.",1,,0,0,0,0,0
1,1,N76.1,Подострый и хронический вагинит,45,2,,,считает себя больной на протяжении многих лет. Появилась боль и скованность в спине. По словам паицентки в настоящее время занимается конным спортом и появляется боль и скованность в поясничном отделе позвончника. В ближайшее время пациентка планирует беременность и поэтому обратилась к неврологу.,,1,считает себя больной на протяжении многих лет появилась боль и скованность в спине по словам паицентки в настоящее время занимается конным спортом и появляется боль и скованность в поясничном отделе позвончника в ближайшее время пациентка планирует беременность и поэтому обратилась к неврологу,2,4,0,0,2
2,2,L23.9,"Аллергический контактный дерматит, причина не уточнена",63,2,,,"Болен около 2-х дней, когда появились описанные жалобы. Самостоятельно не лечился. обратился в МХЦ.",,1,болен около х дней когда появились описанные жалобы самостоятельно не лечился обратился в мхц,3,0,0,2,0
3,3,N48.1,Баланопостит,43,2,,не отягощен,,,1,,0,0,0,0,0
4,4,Z00.0,Общий медицинский осмотр,29,1,,,провела полное обследование для оперативного лечения,,1,провела полное обследование для оперативного лечения,1,0,0,0,1


In [10]:
X_text = data['text']
corpus = X_text.values
vectorizer = CountVectorizer(min_df = 2)
D = vectorizer.fit(corpus)
len(D.vocabulary_)

44144

In [11]:
D.vocabulary_

{'считает': 37463,
 'себя': 33778,
 'больной': 2695,
 'на': 18928,
 'протяжении': 30605,
 'многих': 18293,
 'лет': 15617,
 'появилась': 28341,
 'боль': 2671,
 'скованность': 34580,
 'спине': 35987,
 'по': 26262,
 'словам': 34852,
 'паицентки': 24448,
 'настоящее': 19755,
 'время': 4846,
 'занимается': 10541,
 'конным': 13920,
 'спортом': 36120,
 'появляется': 28393,
 'поясничном': 28461,
 'отделе': 23683,
 'позвончника': 27023,
 'ближайшее': 2440,
 'пациентка': 24890,
 'планирует': 25978,
 'беременность': 2085,
 'поэтому': 28320,
 'обратилась': 21987,
 'неврологу': 20089,
 'болен': 2622,
 'около': 22630,
 'дней': 9019,
 'когда': 13439,
 'появились': 28352,
 'описанные': 22970,
 'жалобы': 9793,
 'самостоятельно': 33242,
 'не': 19966,
 'лечился': 15854,
 'обратился': 22004,
 'мхц': 18835,
 'провела': 29762,
 'полное': 27316,
 'обследование': 22136,
 'для': 8972,
 'оперативного': 22903,
 'лечения': 15792,
 'проводит': 29820,
 'лечение': 15733,
 'прибыла': 29055,
 'коррекции': 14386,
 'наз

In [12]:
morph = pymorphy2.MorphAnalyzer()

In [13]:
stop_eng = nltk.corpus.stopwords.words('english')
stop_rus = nltk.corpus.stopwords.words('russian')

In [14]:
all_text = np.concatenate(data['text'].apply (lambda s: s.strip().split()))
all_words = set(all_text)

In [15]:
print('number all words:', len(all_words))

number all words: 95559


In [16]:
count = pd.Series(all_text).value_counts()
count

в                                     206707
с                                     119749
на                                     89960
не                                     83702
по                                     69093
и                                      52113
жалобы                                 38504
боли                                   36993
г                                      34805
течение                                34393
к                                      32557
около                                  30885
фоне                                   30328
лет                                    30240
лечения                                27852
назад                                  26647
беспокоят                              25685
после                                  24727
отмечает                               24686
самостоятельно                         23158
дней                                   21289
без                                    21152
для       

In [17]:
count.value_counts().head(20)

1     51155
2     12865
3      5922
4      3452
5      2432
6      1791
7      1383
8      1107
9       980
10      752
11      674
12      592
13      550
14      485
16      378
15      377
17      371
18      331
20      294
19      286
dtype: int64

In [18]:
#удалим слова только из согласных
t1 = time.time()
def norm_words(text):
    
    # будем смотреть есть ли гласные
    vowels = 'аеёиоуыюя'
    
    result = []
    for word in text.split():
        for char in word:
            if (char.isalpha() and char[0] in vowels):
                result.append(word)
                break
                              
    return ' '.join(result)

norm_words = pd.Series(all_text).apply(norm_words)

t2 = time.time()
print((t2 - t1)/60)

0.09141053756078084


In [19]:
without_stop = norm_words[~norm_words.isin(stop_rus)]

In [20]:
#удалим очень короткие слова
def long_words(text, n):

    result = []
    for word in text.split():
        if len(word) >= n:
            #print(word)
            result.append(word)
                              
    return ' '.join(result)

In [21]:
#подаем на вход слова и мин. длину
no_short = without_stop.apply(long_words, args=(3,))

In [22]:
#вместо 1 можно поставить любое число, которое мы бцдем считать слишком редкое
without_rare = no_short[no_short.isin(no_short.value_counts()[no_short.value_counts() > 1].index)]

In [23]:
no_rare_set = set(without_rare)

In [24]:
#создадим столбец с норм словами
t1 = time.time()
def cond(text, condition):
    result = []
    for word in text.split():
        if word in condition:
            #print(word)
            result.append(word)
            #print(result)
                              
    return ' '.join(result)

df['without rare'] = df['text'].apply(cond, args = (no_rare_set,))

t2 = time.time()
print((t2 - t1)/60)

0.029169392585754395


In [25]:
df.tail(2)

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,text,past,pres,futr,male,female,without rare
283084,283084,J00,Острый назофарингит (насморк),33,2,,,"обратилась для осмотра при беременности 35 недель. Течение беременности с угрозой прерывания в последний месяц на фоне повышенного тонуса матки, ЖДА в августе, принимала ферро-фольгамму месяц. . За время беременности набрала около 9-ти кг веса. В анамнезе- ДЖВП на фоне дисформии ЖП, хр. гастродуоденит",,1,обратилась для осмотра при беременности недель течение беременности с угрозой прерывания в последний месяц на фоне повышенного тонуса матки жда в августе принимала феррофольгамму месяц за время беременности набрала около ти кг веса в анамнезе джвп на фоне дисформии жп хр гастродуоденит,3,0,0,0,3,обратилась осмотра беременности недель течение беременности угрозой прерывания последний месяц фоне повышенного тонуса матки жда августе принимала феррофольгамму месяц время беременности набрала около веса анамнезе фоне дисформии гастродуоденит
283085,283085,N11,Хронический тубулоинтерстициальный нефрит,42,2,,,без дополнений.,,1,без дополнений,0,0,0,0,0,дополнений


In [26]:
# и лемматизируем его
t1 = time.time()
def lemms(text):
 
    result = []
    for word in text.split():
        # лемматизируем
        result.append(morph.parse(word)[0].normal_form)
                              
    return ' '.join(result)

df['lemm'] = df['without rare'].apply(lemms)

t2 = time.time()
print((t2 - t1)/60)

10.83245636622111


In [27]:
df.head(2)

,ID,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit,text,past,pres,futr,male,female,without rare,lemm
0,0,J06.0,Острый ларингофарингит,29,2,,,,"На жевательной поверхности 2.6- кариозная полость средней глубины, выполненная пигментированным дентином. Зондирование слегка болезненно по эмалево-дентинной границе. Реакция на температурные раздражители отсутствует.Перкуссия б/б.",1,,0,0,0,0,0,,
1,1,N76.1,Подострый и хронический вагинит,45,2,,,считает себя больной на протяжении многих лет. Появилась боль и скованность в спине. По словам паицентки в настоящее время занимается конным спортом и появляется боль и скованность в поясничном отделе позвончника. В ближайшее время пациентка планирует беременность и поэтому обратилась к неврологу.,,1,считает себя больной на протяжении многих лет появилась боль и скованность в спине по словам паицентки в настоящее время занимается конным спортом и появляется боль и скованность в поясничном отделе позвончника в ближайшее время пациентка планирует беременность и поэтому обратилась к неврологу,2,4,0,0,2,считает больной протяжении многих лет появилась боль скованность спине словам паицентки настоящее время занимается конным спортом появляется боль скованность поясничном отделе позвончника ближайшее время пациентка планирует беременность поэтому обратилась неврологу,считать больной протяжение многий год появиться боль скованность спина слово паицентка настоящее время заниматься конный спорт появляться боль скованность поясничный отдел позвончник ближний время пациентка планировать беременность поэтому обратиться невролог


In [32]:
df.to_csv('lemm.csv', index = False)

In [28]:
all_lemm = np.concatenate(df['lemm'].apply (lambda s: s.strip().split()))
bugwords_lemm = pd.Series(all_lemm).unique()

In [29]:
pd.Series(all_lemm).to_csv(r'all_words.txt', header=None, index=None, sep=' ', mode='a')

In [30]:
pd.Series(bugwords_lemm).to_csv(r'bugwords_lemm.txt', header=None, index=None, sep=' ', mode='a')

In [115]:
print(len(all_words_lemm), len(all_lemm))

63589 4322030


In [155]:
print(len(all_words_lemm), len(all_lemm), len(lemms))

24248 3137151 24248


In [109]:
count1 = pd.Series(all_lemm).value_counts()
count1

боль                       61111
год                        56010
лечение                    51577
течение                    45809
жалоба                     45729
день                       38762
беспокоить                 35905
обследование               35177
неделя                     31409
отмечать                   31087
около                      30885
фон                        30719
обратиться                 28856
принимать                  28576
назад                      26651
месяц                      25536
последний                  23612
лечиться                   23296
самостоятельно             23158
появиться                  21441
терапия                    19817
связывать                  19372
провести                   18988
консультация               18065
результат                  17925
дать                       17350
приём                      16238
больной                    15959
эффект                     15719
обращаться                 15701
          

In [141]:
txt = df['lemm'].str.cat(sep=' ')
words = nltk.tokenize.word_tokenize(txt)
word_dist = nltk.FreqDist(words)

stopwords = nltk.corpus.stopwords.words('russian')
words_except_stop_dist = nltk.FreqDist(w for w in words if w not in stopwords) 

print('All frequencies, including STOPWORDS:')
print('=' * 60)
rslt = pd.DataFrame(word_dist.most_common(70),
                    columns=['Word', 'Frequency'])
print(rslt)
print('=' * 60)

rslt = pd.DataFrame(words_except_stop_dist.most_common(70),
                    columns=['Word', 'Frequency']).set_index('Word')

matplotlib.style.use('ggplot')

rslt.plot.bar(rot=0, figsize=(18, 16))

All frequencies, including STOPWORDS:
              Word  Frequency
0             боль      61111
1              год      56010
2          лечение      51577
3          течение      45809
4           жалоба      45729
5             день      38762
6       беспокоить      35905
7     обследование      35177
8           неделя      31409
9         отмечать      31087
10           около      30885
11             фон      30719
12      обратиться      28856
13       принимать      28576
14           назад      26651
15           месяц      25536
16       последний      23612
17        лечиться      23296
18  самостоятельно      23158
19       появиться      21441
20         терапия      19817
21       связывать      19372
22        провести      18988
23    консультация      18065
24       результат      17925
25            дать      17350
26           приём      16238
27         больной      15959
28          эффект      15719
29      обращаться      15701
..             ...        ...
40

AttributeError: module 'matplotlib' has no attribute 'style'

In [150]:
"""batch_vectorizer = artm.BatchVectorizer(data_path=df['lemm'], data_format='bow_uci',
                                        collection_name='kos', target_folder='kos_batches')"""

"batch_vectorizer = artm.BatchVectorizer(data_path=df['lemm'], data_format='bow_uci',\n                                        collection_name='kos', target_folder='kos_batches')"

In [33]:
dictionary = artm.Dictionary()

model_plsa = artm.ARTM(topic_names=['topic_{}'.format(i) for i in range(15)],
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                                                    dictionary=dictionary)],
                       cache_theta=True)

model_artm = artm.ARTM(topic_names=['topic_{}'.format(i) for i in range(15)],
                       scores=[artm.PerplexityScore(name='PerplexityScore',
                                                    dictionary=dictionary)],
                       regularizers=[artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=-0.15)],
                       cache_theta=True)

In [149]:
"""batch_vectorizer = artm.BatchVectorizer(data_path='my_collection_batches',
                                        data_format='dataframe')"""

"batch_vectorizer = artm.BatchVectorizer(data_path='my_collection_batches',\n                                        data_format='dataframe')"

In [34]:
import re
from bs4 import BeautifulSoup   

C:\ProgramData\Anaconda3\lib\site-packages\bs4\element.py:15: DeprecationWarning: invalid escape sequence \s
  whitespace_re = re.compile("\s+")
C:\ProgramData\Anaconda3\lib\site-packages\bs4\element.py:1421: DeprecationWarning: invalid escape sequence \d
  pseudo_attributes = re.match('([a-zA-Z\d-]+)\(([a-zA-Z\d]+)\)', pseudo)
C:\ProgramData\Anaconda3\lib\site-packages\bs4\dammit.py:49: DeprecationWarning: invalid escape sequence \?
  '^<\?.*encoding=[\'"](.*?)[\'"].*\?>'.encode(), re.I)


In [35]:
          

# Initialize the BeautifulSoup object on a single movie review     
example1 = BeautifulSoup(df["lemm"][1])  

# Print the raw review and then the output of get_text(), for 
# comparison
print (df["lemm"][1])
print (example1.get_text())

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)


считать больной протяжение многий год появиться боль скованность спина слово паицентка настоящее время заниматься конный спорт появляться боль скованность поясничный отдел позвончник ближний время пациентка планировать беременность поэтому обратиться невролог
считать больной протяжение многий год появиться боль скованность спина слово паицентка настоящее время заниматься конный спорт появляться боль скованность поясничный отдел позвончник ближний время пациентка планировать беременность поэтому обратиться невролог


In [42]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
train_data_features = vectorizer.fit_transform(all_lemm[:800000])

train_data_features = train_data_features.toarray()

MemoryError: 

In [ ]:
train_data_features.shape

In [ ]:
vocab = vectorizer.get_feature_names()
print (vocab)

In [ ]:
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print (count, tag)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

In [ ]:

# Create an empty list and append the clean reviews one by one
num_reviews = len(df["lemm"])
clean_test_reviews = [] 

print "Cleaning and parsing the test set movie reviews...\n"
for i in range(0,num_reviews):
    if( (i+1) % 1000 == 0 ):
        print "Review %d of %d\n" % (i+1, num_reviews)
    clean_review = review_to_words( df["lemm"][i] )
    clean_test_reviews.append( clean_review )

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )